In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'microsoft-catsvsdogs-dataset' dataset.
Path to dataset files: /kaggle/input/microsoft-catsvsdogs-dataset


In [3]:
import os
print(os.listdir(path))

['PetImages', 'readme[1].txt', 'MSR-LA - 3467.docx']


In [4]:
train_dir = os.path.join(path, 'PetImages')


In [5]:
import shutil
import os

dst_path = '/content/my_dataset'
if os.path.exists(dst_path):
    shutil.rmtree(dst_path)
shutil.copytree(path, dst_path)

'/content/my_dataset'

In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Define the transformation pipeline
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to a consistent size
    transforms.ToTensor(),           # Convert to tensor [0, 1]
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Standard normalization
])

In [7]:
import os
from PIL import Image

data_dir = '/content/my_dataset/PetImages' # or your specific path
categories = ['Cat', 'Dog']

for category in categories:
    folder_path = os.path.join(data_dir, category)
    removed_count = 0

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            # Try to open the image and verify it
            with Image.open(img_path) as img:
                img.verify()
        except (IOError, SyntaxError, Image.UnidentifiedImageError) as e:
            # If PIL can't open it, delete it
            os.remove(img_path)
            removed_count += 1

    print(f"Removed {removed_count} corrupted images from {category} folder.")

Removed 2 corrupted images from Cat folder.


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Removed 2 corrupted images from Dog folder.


In [8]:
# The 'path' variable comes from your kagglehub download

data_dir = os.path.join(dst_path, 'PetImages')

# Load the dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Split into Train and Validation (80/20 split)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(f"Total images: {len(full_dataset)}")
print(f"Classes found: {full_dataset.classes}")

Total images: 24998
Classes found: ['Cat', 'Dog']


In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Check a batch
images, labels = next(iter(train_loader))
print(f"Batch shape: {images.shape}") # Should be [32, 3, 128, 128]

Batch shape: torch.Size([32, 3, 128, 128])


In [1]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, 2) # Assuming 128x128 input

        # Custom weight initialization
        nn.init.xavier_uniform_(self.conv1.weight) # Xavier uniform for convolutional weights
        nn.init.constant_(self.conv1.bias, 0.01)   # Small constant for convolutional biases
        nn.init.xavier_uniform_(self.fc1.weight)    # Xavier uniform for fully connected weights
        nn.init.constant_(self.fc1.bias, 0)        # Zeros for fully connected biases

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 64 * 64)
        x = self.fc1(x)
        return x

model = SimpleCNN().to('cuda') # Use GPU if available

# print("Sample of initialized conv1 weights (first row):")
# print(model.conv1.weight[0, 0, :, :])
# print("Sample of initialized conv1 bias:")
# print(model.conv1.bias)


In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.7820382914066315
Epoch 2, Loss: 0.505348783826828
Epoch 3, Loss: 0.4372052148103714
Epoch 4, Loss: 0.38380809634923935
Epoch 5, Loss: 0.33164840748310087
Epoch 6, Loss: 0.28701964209079744
Epoch 7, Loss: 0.24679675822257996
Epoch 8, Loss: 0.21481974304318427
Epoch 9, Loss: 0.1835541669487953
Epoch 10, Loss: 0.16260048270821573


In [12]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad(): # Disable gradient calculation for speed/memory
    for images, labels in val_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {100 * correct / total:.2f}%')

Accuracy on validation set: 69.86%


In [15]:
torch.save(model.state_dict(), 'cats_dogs_model_1.pth')
print("Model saved as cats_dogs_model_1.pth")

Model saved as cats_dogs_model_1.pth


In [16]:
class SimpleCNN2(nn.Module):
    def __init__(self):
        super().__init__() # Changed from super(SimpleCNN, self) to super()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, 2) # Assuming 128x128 input

        # Custom weight initialization
        nn.init.kaiming_normal_(self.conv1.weight) # kaiming_normal_ for convolutional weights
        nn.init.constant_(self.conv1.bias, 0.01)   # Small constant for convolutional biases
        nn.init.kaiming_normal_(self.fc1.weight)    # Kaiming normal for fully connected weights
        nn.init.constant_(self.fc1.bias, 0)        # Zeros for fully connected biases

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 64 * 64)
        x = self.fc1(x)
        return x

model = SimpleCNN2().to('cuda')


criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 1.2179470736503601
Epoch 2, Loss: 1.217990549659729
Epoch 3, Loss: 1.2179853203773499
Epoch 4, Loss: 1.2179677515029907
Epoch 5, Loss: 1.2179656868934632
Epoch 6, Loss: 1.218007322216034
Epoch 7, Loss: 1.2179712107658387
Epoch 8, Loss: 1.2179925091743469
Epoch 9, Loss: 1.217990859222412
Epoch 10, Loss: 1.2179740591049195


In [17]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad(): # Disable gradient calculation for speed/memory
    for images, labels in val_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {100 * correct / total:.2f}%')

Accuracy on validation set: 47.90%


In [18]:
torch.save(model.state_dict(), 'cats_dogs_model_2.pth')
print("Model saved as cats_dogs_model_2.pth")

Model saved as cats_dogs_model_2.pth


In [20]:
class SimpleCNN3(nn.Module):
    def __init__(self):
        super().__init__() # Changed from super(SimpleCNN, self) to super()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, 2) # Assuming 128x128 input

        # Custom weight initialization
        nn.init.uniform_(self.conv1.weight) # random for convolutional weights
        nn.init.constant_(self.conv1.bias, 0.01)   # Small constant for convolutional biases
        nn.init.uniform_(self.fc1.weight)    # random for fully connected weights
        nn.init.constant_(self.fc1.bias, 0)        # Zeros for fully connected biases

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 64 * 64)
        x = self.fc1(x)
        return x

model = SimpleCNN3().to('cuda')


criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad(): # Disable gradient calculation for speed/memory
    for images, labels in val_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {100 * correct / total:.2f}%')

torch.save(model.state_dict(), 'cats_dogs_model_3.pth')
print("Model saved as cats_dogs_model_3.pth")

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 492.3087629394531
Epoch 2, Loss: 492.3176787597656
Epoch 3, Loss: 492.31218322753904
Epoch 4, Loss: 492.31887907714844
Epoch 5, Loss: 492.3440726074219
Epoch 6, Loss: 492.3090771240234
Epoch 7, Loss: 492.316772265625
Epoch 8, Loss: 492.30397082519534
Epoch 9, Loss: 492.310807421875
Epoch 10, Loss: 492.32620485839846
Accuracy on validation set: 48.14%
Model saved as cats_dogs_model_3.pth


In [21]:
class SimpleCNN3(nn.Module):
    def __init__(self):
        super().__init__() # Changed from super(SimpleCNN, self) to super()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, 2) # Assuming 128x128 input

        # Custom weight initialization
        nn.init.xavier_uniform_(self.conv1.weight) # Xavier uniform for convolutional weights
        nn.init.constant_(self.conv1.bias, 0.01)   # Small constant for convolutional biases
        nn.init.xavier_uniform_(self.fc1.weight)    # Xavier uniform for fully connected weights
        nn.init.constant_(self.fc1.bias, 0)        # Zeros for fully connected biases

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = x.view(-1, 16 * 64 * 64)
        x = self.fc1(x)
        return x

model = SimpleCNN3().to('cuda')


criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad(): # Disable gradient calculation for speed/memory
    for images, labels in val_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {100 * correct / total:.2f}%')

torch.save(model.state_dict(), 'cats_dogs_model_4.pth')
print("Model saved as cats_dogs_model_4.pth")

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 0.7716261992454528
Epoch 2, Loss: 0.7716352178573609
Epoch 3, Loss: 0.7716429697036743
Epoch 4, Loss: 0.7716566767692566
Epoch 5, Loss: 0.7716396781921386
Epoch 6, Loss: 0.771645639514923
Epoch 7, Loss: 0.7716466102600098
Epoch 8, Loss: 0.7716590131759643
Epoch 9, Loss: 0.7716462043762207
Epoch 10, Loss: 0.7716323496818542
Accuracy on validation set: 48.96%
Model saved as cats_dogs_model_3.pth


In [22]:
torch.save(model.state_dict(), 'cats_dogs_model_4.pth')

In [24]:
class SimpleCNN3(nn.Module):
    def __init__(self):
        super().__init__() # Changed from super(SimpleCNN, self) to super()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, 2) # Assuming 128x128 input

        # Custom weight initialization
        nn.init.xavier_uniform_(self.conv1.weight) # Xavier uniform for convolutional weights
        nn.init.constant_(self.conv1.bias, 0.01)   # Small constant for convolutional biases
        nn.init.xavier_uniform_(self.fc1.weight)    # Xavier uniform for fully connected weights
        nn.init.constant_(self.fc1.bias, 0)        # Zeros for fully connected biases

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = x.view(-1, 16 * 64 * 64)
        x = self.fc1(x)
        return x

model = SimpleCNN3().to('cuda')


criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad(): # Disable gradient calculation for speed/memory
    for images, labels in val_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {100 * correct / total:.2f}%')

torch.save(model.state_dict(), 'cats_dogs_model_5.pth')
print("Model saved as cats_dogs_model_5.pth")

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 0.8889616750240326
Epoch 2, Loss: 0.8889877446174621
Epoch 3, Loss: 0.8889830102920532
Epoch 4, Loss: 0.8889696611404418
Epoch 5, Loss: 0.8889585578918457
Epoch 6, Loss: 0.8889765306949615
Epoch 7, Loss: 0.888968433380127
Epoch 8, Loss: 0.8889755222320557
Epoch 9, Loss: 0.8889608957290649
Epoch 10, Loss: 0.888961439037323
Accuracy on validation set: 50.14%
Model saved as cats_dogs_model_5.pth


In [25]:
class SimpleCNN3(nn.Module):
    def __init__(self):
        super().__init__() # Changed from super(SimpleCNN, self) to super()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, 2) # Assuming 128x128 input

        # Custom weight initialization
        nn.init.xavier_uniform_(self.conv1.weight) # Xavier uniform for convolutional weights
        nn.init.constant_(self.conv1.bias, 0.01)   # Small constant for convolutional biases
        nn.init.xavier_uniform_(self.fc1.weight)    # Xavier uniform for fully connected weights
        nn.init.constant_(self.fc1.bias, 0)        # Zeros for fully connected biases

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 64 * 64)
        x = self.fc1(x)
        return x

model = SimpleCNN3().to('cuda')


criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.SGD(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad(): # Disable gradient calculation for speed/memory
    for images, labels in val_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {100 * correct / total:.2f}%')

torch.save(model.state_dict(), 'cats_dogs_model_6.pth')
print("Model saved as cats_dogs_model_6.pth")

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1, Loss: 0.8123146061897277
Epoch 2, Loss: 0.8122840463638306
Epoch 3, Loss: 0.812307015132904
Epoch 4, Loss: 0.8123001019477845
Epoch 5, Loss: 0.8122790049552917
Epoch 6, Loss: 0.8122962166786194
Epoch 7, Loss: 0.8122986666679383
Epoch 8, Loss: 0.8122985091209411
Epoch 9, Loss: 0.8122832621574402
Epoch 10, Loss: 0.8122864789009094
Accuracy on validation set: 50.36%
Model saved as cats_dogs_model_6.pth


In [26]:
class SimpleCNN3(nn.Module):
    def __init__(self):
        super().__init__() # Changed from super(SimpleCNN, self) to super()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, 2) # Assuming 128x128 input

        # Custom weight initialization
        nn.init.xavier_uniform_(self.conv1.weight) # Xavier uniform for convolutional weights
        nn.init.constant_(self.conv1.bias, 0.01)   # Small constant for convolutional biases
        nn.init.xavier_uniform_(self.fc1.weight)    # Xavier uniform for fully connected weights
        nn.init.constant_(self.fc1.bias, 0)        # Zeros for fully connected biases

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 64 * 64)
        x = self.fc1(x)
        return x

model = SimpleCNN3().to('cuda')


criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.RMSprop(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to('cuda'), labels.to('cuda')

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad(): # Disable gradient calculation for speed/memory
    for images, labels in val_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {100 * correct / total:.2f}%')

torch.save(model.state_dict(), 'cats_dogs_model_7.pth')
print("Model saved as cats_dogs_model_7.pth")

Epoch 1, Loss: 0.9230966591835021
Epoch 2, Loss: 0.9230959101676941
Epoch 3, Loss: 0.923071548318863
Epoch 4, Loss: 0.9230881919384003
Epoch 5, Loss: 0.9230868430137634
Epoch 6, Loss: 0.9231089612960816
Epoch 7, Loss: 0.9231427976608276
Epoch 8, Loss: 0.9230854653358459
Epoch 9, Loss: 0.9230904733657836
Epoch 10, Loss: 0.9231180364608764
Accuracy on validation set: 50.40%
Model saved as cats_dogs_model_7.pth
